<a href="https://colab.research.google.com/github/knc6/jarvis-tools-notebooks/blob/master/jarvis-tools-notebooks/ocp_load_pretrained_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Use OCP pretrained models for JARVIS-Leaderboard

In [1]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-cluster -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

2.0.1+cu118
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
import os
if not os.path.exists('ocp'):
     !git clone https://github.com/Open-Catalyst-Project/ocp.git
os.chdir('ocp')
!pip install -e .

Obtaining file:///content/ocp
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for ocp-models (pyproject.toml) ... done
  Created wheel for ocp-models: filename=ocp_models-0.0.3-0.editable-py3-none-any.whl size=3295 sha256=806d33fdf8b6ce97314ce678978deb0e20405ab33a22bf097a4670715b614e7c
  Stored in directory: /tmp/pip-ephem-wheel-cache-qpd1jg0i/wheels/29/db/87/61b8ac706262b22238c01b070b675db289c051f2d890c74f3b
Successfully built ocp-models
  Attempting uninstall: ocp-models
    Found existing installation: ocp-models 0.0.3
    Uninstalling ocp-models-0.0.3:
      Successfully uninstalled ocp-models-0.0.3


In [3]:
pip -q install  lmdb pymatgen==2023.5.10 ase==3.22.1 orjson e3nn==0.2.6 wandb

In [4]:
import os, torch
from ase.io import read
from ocpmodels.preprocessing import AtomsToGraphs
from ocpmodels.models import SchNet

In [5]:
!wget https://dl.fbaipublicfiles.com/opencatalystproject/models/2021_02/is2re/schnet_all.pt

--2023-08-15 12:18:00--  https://dl.fbaipublicfiles.com/opencatalystproject/models/2021_02/is2re/schnet_all.pt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.249.85.90, 13.249.85.39, 13.249.85.89, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.249.85.90|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14643781 (14M) [binary/octet-stream]
Saving to: ‘schnet_all.pt.1’

schnet_all.pt.1     100%[===================>]  13.96M  66.3MB/s    in 0.2s    

2023-08-15 12:18:01 (66.3 MB/s) - ‘schnet_all.pt.1’ saved [14643781/14643781]



In [6]:
device = "cpu"
if torch.cuda.is_available():
    device = torch.device("cuda")
attrs = torch.load("schnet_all.pt", map_location=device)["config"][
    "model_attributes"
]

In [7]:
model = SchNet(
    None,
    None,
    1,
    cutoff=6.0,
    hidden_channels=attrs["hidden_channels"],
    num_interactions=attrs["num_interactions"],
    num_gaussians=attrs["num_gaussians"],
    regress_forces=False,
    use_pbc=True,
)

state_dict = torch.load("schnet_all.pt", map_location=device)["state_dict"]
state_dict = {k[2 * len("module.") :]: v for k, v in state_dict.items()}
model.load_state_dict(state_dict, strict=True)
model.to(device)

SchNetWrap(hidden_channels=384, num_filters=128, num_interactions=4, num_gaussians=100, cutoff=6.0)

In [8]:
import os
os.chdir('/content')
if not os.path.exists('jarvis_leaderboard'):
  !git clone https://github.com/usnistgov/jarvis_leaderboard.git
os.chdir('jarvis_leaderboard')
!pip install -e .

Obtaining file:///content/jarvis_leaderboard
  Preparing metadata (setup.py) ... done
  Attempting uninstall: jarvis-leaderboard
    Found existing installation: jarvis-leaderboard 2023.8.1
    Uninstalling jarvis-leaderboard-2023.8.1:
      Successfully uninstalled jarvis-leaderboard-2023.8.1
  Running setup.py develop for jarvis-leaderboard


In [15]:
from jarvis.core.atoms import Atoms
from jarvis.core.specie import atomic_numbers_to_symbols
import os, torch
from ase.io import read
from torch.nn.parallel import DistributedDataParallel as DDP
import torch.distributed as dist
from jarvis.core.atoms import Atoms
from tqdm import tqdm
from jarvis.db.figshare import  get_request_data
import json,zipfile
import numpy as np
import pandas as pd
from jarvis.db.jsonutils import loadjson,dumpjson
import os
from jarvis.core.atoms import Atoms
from jarvis.core.atoms import Atoms
from jarvis.core.specie import atomic_numbers_to_symbols
from ocpmodels.datasets import SinglePointLmdbDataset
import os, torch
from ase.io import read
from ocpmodels.preprocessing import AtomsToGraphs
from ocpmodels.models import CGCNN,SchNet
from torch.nn.parallel import DistributedDataParallel as DDP
import torch.distributed as dist
from jarvis.core.atoms import Atoms
from ocpmodels.datasets import data_list_collater
from jarvis.db.figshare import data
import pandas as pd
dat = data('ocp10k')
df = pd.DataFrame(dat)

path = "/content/jarvis_leaderboard/jarvis_leaderboard/benchmarks/AI/SinglePropertyPrediction/ocp10k_relaxed_energy.json.zip"
js_tag = "ocp10k_relaxed_energy.json"
id_data = json.loads(zipfile.ZipFile(path).read(js_tag))
train_ids = np.array(list(id_data['train'].keys()))
val_ids = np.array(list(id_data['val'].keys()))
test_ids = np.array(list(id_data['test'].keys()))
train_df = df[df['id'].isin(train_ids)]
val_df = (df[df['id'].isin(val_ids)])#[:take_val]
test_df = (df[df['id'].isin(test_ids)])

Obtaining OCP 10k train dataset, 59886...
Reference:https://github.com/Open-Catalyst-Project/ocp
Loading the zipfile...
Loading completed.


In [16]:
model.eval()

SchNetWrap(hidden_channels=384, num_filters=128, num_interactions=4, num_gaussians=100, cutoff=6.0)

In [ ]:
%%time
a2g = AtomsToGraphs(
    max_neigh=200,
    radius=6,
    r_energy=False,
    r_forces=False,
    r_distances=True,
    r_edges=True,
)
target_mean= -1.525913953781128
target_std= 2.279365062713623
f=open('AI-SinglePropertyPrediction-relaxed_energy-ocp10k-test-mae.csv','w')
f.write('id,target,prediction\n')

for ii,i in tqdm(test_df.iterrows()):
    fname=i['id']
    atoms=(Atoms.from_dict(i['atoms'])).ase_converter()
    actual=i['relaxed_energy']
    relaxed_energy = (actual-target_mean)/target_std
    scaled=relaxed_energy
    data = a2g.convert(atoms).to(device)
    batch = data_list_collater([data], otf_graph=False)
    out = model(batch)
    pred=(out[0].detach().cpu().numpy().flatten().tolist()[0])*target_std+target_mean
    line=str(fname)+','+str(actual)+','+str(pred) #+'\n'
    #line=str(i.sid)+','+str(actual)+','+str(scaled)+','+str(pred) #+'\n'
    f.write(line+'\n')
f.close()

1094it [02:45,  5.84it/s]

In [ ]:
from ocpmodels.common.relaxation.ase_utils import OCPCalculator
import os
os.chdir('/content/ocp')
!wget -q https://dl.fbaipublicfiles.com/opencatalystproject/models/2021_08/s2ef/gemnet_t_direct_h512_all.pt
checkpoint_path = "/content/ocp/gemnet_t_direct_h512_all.pt"
config_yml_path = "configs/s2ef/all/gemnet/gemnet-dT.yml"

# Define the calculator
calc = OCPCalculator(config_yml=config_yml_path, checkpoint=checkpoint_path)